In [84]:
from pycocotools.coco import COCO
from sklearn import tree, metrics
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.util import crop
from skimage import io, color
from math import floor, ceil
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.decomposition import TruncatedSVD
import json
from LoadLocalCOCO import LoadLocalCOCO as llc
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline


In [106]:
# Save the model
import pickle
from joblib import dump, load

In [2]:
def flatten_list(qlist):
    return [y for x in qlist for y in x]

In [3]:
train_instance="annotations/instances_train2017.json"
val_instance="annotations/instances_val2017.json"


In [4]:
coco_path = "D:/FSR/COCO/train2017"

In [5]:
classes = ['dog', 'cat']

In [6]:
coco_train = llc(train_instance, coco_path)
coco_val = llc(val_instance, coco_path)

In [7]:
# train_n = [coco_train.get_number_of_images_per_category(x) for x in classes]
# val_n = [coco_val.get_number_of_images_per_category(x) for x in classes]
# print(f'Training:> \t Dog: {train_n[0]} \t Cat: {train_n[1]}')
# print(f'Validation:> \t Dog: {val_n[0]} \t Cat: {val_n[1]}')

In [8]:
data_train = []
for category in classes:
    data = coco_train.get_cropped_images_flatten_and_resized(category, resize=(64, 64))
    data_train.append(data)

5508it [05:51, 15.69it/s]
2it [00:00, 16.54it/s]Broken images: [7125, 16775, 24247, 24247, 55296, 47386, 69468, 35613, 77325, 12696, 22427, 26321, 54957, 790, 1360, 55478, 42341, 57631, 2400, 10276, 94049, 58741, 5782, 44613, 58741, 24159, 44613, 29176] images
4768it [04:57, 16.02it/s]Broken images: [5617, 711, 39171, 9413, 46749, 18211, 26375, 26375, 26375, 66041] images



In [9]:
np.shape(data_train[0])

(5480, 12288)

In [10]:
labels = len(data_train[0])*[1]+len(data_train[1])*[0]

In [11]:
np.shape(labels)

(10238,)

In [12]:
np.shape(data_train)

(2,)

In [13]:
data_train = flatten_list(data_train)

In [14]:
np.shape(data_train)

(10238, 12288)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data_train, labels)

In [18]:
pca = PCA(n_components = 25)
pca.fit(X_train)
reduced_X_train_pca, reduced_X_test_pca = pca.transform(X_train), pca.transform(X_test)

In [28]:
np.shape(reduced_X_train)

(7678, 24)

In [29]:
np.shape(X_train)

(7678, 67500)

In [69]:
ica = FastICA(n_components=25, random_state=0, whiten=True)

In [72]:
reduced_X_train_ica = ica.transform(X_train)

In [73]:
reduced_X_test_ica = ica.transform(X_test)

In [16]:
svd = TruncatedSVD(n_components = 25)
svd.fit(X_train)

TruncatedSVD(algorithm='randomized', n_components=25, n_iter=5,
             random_state=None, tol=0.0)

In [17]:
reduced_X_train_svd, reduced_X_test_svd = svd.transform(X_train), svd.transform(X_test)

In [107]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(reduced_X_train_pca, y_train)

In [108]:
y_pred = clf.predict(reduced_X_test_pca)

In [109]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred,  target_names=['Dog','Cat']))

[[614 596]
 [595 755]]
              precision    recall  f1-score   support

         Dog       0.51      0.51      0.51      1210
         Cat       0.56      0.56      0.56      1350

    accuracy                           0.53      2560
   macro avg       0.53      0.53      0.53      2560
weighted avg       0.53      0.53      0.53      2560



In [110]:
clf.score(reduced_X_test_pca, y_test)

0.534765625

In [111]:
# Save model

# pca_dtl = 'pca_n25_dct_default.pkl'
# with open(pca_dtl, 'wb') as file:
#     pickle.dump(clf, file)

# pakai joblib
pca_dtl = 'pca_n25_dct_default.joblib'
dump(clf, pca_dtl) 

['pca_n25_dct_default.joblib']

In [112]:
clf_ica = tree.DecisionTreeClassifier()
clf_ica = clf.fit(reduced_X_train_ica, y_train)

In [113]:
y_pred_ica = clf_ica.predict(reduced_X_test_ica)

In [114]:
print(confusion_matrix(y_test, y_pred_ica))
print(classification_report(y_test, y_pred_ica,  target_names=['Dog','Cat']))

[[595 615]
 [570 780]]
              precision    recall  f1-score   support

         Dog       0.51      0.49      0.50      1210
         Cat       0.56      0.58      0.57      1350

    accuracy                           0.54      2560
   macro avg       0.53      0.53      0.53      2560
weighted avg       0.54      0.54      0.54      2560



In [115]:
clf_ica.score(reduced_X_test_ica, y_test)

0.537109375

In [116]:
# Save model

# ica_dtl = 'ica_n25_dct_default.pkl'
# with open(ica_dtl, 'wb') as file:
#     pickle.dump(clf_ica, file)

# pakai joblib
ica_dtl = 'ica_n25_dct_default.joblib'
dump(clf_ica, ica_dtl) 

['ica_n25_dct_default.joblib']

In [117]:
clf_svd = tree.DecisionTreeClassifier()
clf_svd = clf.fit(reduced_X_train_svd, y_train)

In [118]:
y_pred_svd = clf_svd.predict(reduced_X_test_svd)

In [119]:
print(confusion_matrix(y_test, y_pred_svd))
print(classification_report(y_test, y_pred_svd,  target_names=['Dog','Cat']))

[[590 620]
 [550 800]]
              precision    recall  f1-score   support

         Dog       0.52      0.49      0.50      1210
         Cat       0.56      0.59      0.58      1350

    accuracy                           0.54      2560
   macro avg       0.54      0.54      0.54      2560
weighted avg       0.54      0.54      0.54      2560



In [120]:
clf_svd.score(reduced_X_test_svd, y_test)

0.54296875

In [121]:
# Save model

# svd_dtl = 'svd_n25_dct_default.pkl'
# with open(svd_dtl, 'wb') as file:
#     pickle.dump(clf_svd, file)

# pakai joblib
svd_dtl = 'svd_n25_dct_default.joblib'
dump(clf_svd, svd_dtl) 

['svd_n25_dct_default.joblib']

In [122]:
np.shape(reduced_X_train_pca[0])

(25,)

In [123]:
np.shape(data_train)

(10238, 12288)

### Features
1. Color -> color moment
2. texture -> HOG/Gabor
3. GLCM -> Some Features

PCA bisa jadi ga perlu but who knows